# Think Act Agent

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../../../')

In [2]:
from typing import List, Optional, Any
from langchain.tools import tool, Tool
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.chat_models.anthropic import ChatAnthropic
from langchain.llms import OpenAI
from langchain.automaton.agent_implementations import think_act_agent

In [3]:
def get_tools() -> List[Tool]:
    @tool
    def name() -> str:
        """Use to look up the user's name"""
        return "Eugene"

    @tool
    def get_weather(city: str) -> str:
        """Get weather in a specific city."""
        return "42F and sunny"

    @tool
    def add(x: int, y: int) -> int:
        """Use to add two numbers. For example; { "x": 2, "y": 10}"""
        return x + y

    @tool
    def divide(x: float, y: float) -> float:
        """Divide x by y"""
        return x / y

    return list(locals().values())


tools = get_tools()

In [4]:
llm = OpenAI(temperature=0)

In [5]:
agent = think_act_agent.ThinkActAgent(llm, tools)

In [6]:
start_messages = think_act_agent.generate_memory(tools)

In [9]:
from langchain.callbacks.manager import (
    trace_as_chain_group,
    atrace_as_chain_group,
)


In [12]:
question = HumanMessage(content="what is 12381237/23? What is my name? And what is the weather in new york?")
messages = start_messages + [question]

with trace_as_chain_group("think_act_agent") as group_manager:
    for msg in agent.run(messages, config={"callbacks": group_manager}):
        print(msg)

ai:  I need to use the divide tool and the get_weather tool
<action>
{
  "action": "divide",
  "action_input": { "x": 12381237, "y": 23 }
}
</action>

FunctionCall(name=divide, named_arguments={'x': 12381237, 'y': 23})
FunctionResult(name=divide, result=538314.6521739131, error=None)
ai:  I need to use the name tool and the get_weather tool
<action>
{
  "action": "name",
  "action_input": ""
}
</action>

FunctionCall(name=name, named_arguments={})
FunctionResult(name=name, result=Eugene, error=None)
ai: I need to use the get_weather tool
<action>
{
  "action": "get_weather",
  "action_input": "New York"
}
</action>

AdHocMessage(type=error, data=Invalid action blob 
{
  "action": "get_weather",
  "action_input": "New York"
}
, action_input must be a dict)
ai: 
Thought:I need to use the get_weather tool
<action>
{
  "action": "get_weather",
  "action_input": {"city": "New York"}
}
</action>

FunctionCall(name=get_weather, named_arguments={'city': 'New York'})
FunctionResult(name=get_wea